In [1]:
import numpy as np
import tbmodels
import h5py

In [2]:
def apply_efield(h0,efield,z_pos):
    h = h0.copy()
    for i in np.arange(h0.shape[0]):
        h[i,i] = h0[i,i]+efield*z_pos[i]
    return h

def find_efermi_bands(eigvals_all, nfilling):
    vbm = eigvals_all[:,nfilling-1].max()    
    cbm = eigvals_all[:,nfilling].min()
    return vbm, cbm, (vbm+cbm)/2

In [3]:
nwann = 56
nfilling = 36

wpath = f"../tb_model/model_o{nwann}/"

model = tbmodels.Model.from_wannier_files(
            hr_file = f'{wpath}/wte2.o{nwann}_hr.dat',
            xyz_file = f'{wpath}/wte2.o{nwann}_centres.xyz',
            win_file = f'{wpath}/wte2.o{nwann}.win')

# apply on-site electric fields to the wannier orbitals
c = 6.576*4.0316*0.529177
z_pos = model.pos[:,2]*c

z_pos_sym = z_pos.copy()
for iz in np.arange(z_pos.shape[0]):
    z_pos_sym[iz] = 0.5*(z_pos[iz]+z_pos[iz+(-1)**(iz%2)])

In [4]:
# generate a list of k points along the -X-Gamma-X path
nk = 101
kx_list = np.linspace(-0.5,0.5,nk)
k_list = [[kx,0,0] for kx in kx_list]

In [5]:
# generate the band structure on a given k-path for the following efields
efields = [-0.5, -0.35, -0.3, -0.2, 0, 0.2, 0.3, 0.35, 0.5]

with h5py.File("./data/bulk_bands_efields.h5", 'w') as f:
    f.create_dataset("kx_list", data=kx_list)
    for efield in efields:
        h0_k = model.hamilton(k_list)
        h_k = [apply_efield(h0,efield,z_pos) for h0 in h0_k]
        eigvals, eigvecs = np.linalg.eigh(h_k)
        vbm, cbm, efermi = find_efermi_bands(eigvals, nfilling)
        print("efield, efermi:", efield, efermi)
        # save the eigvals and eigvecs to the file
        f.create_dataset(f"e{efield}/eigvals", data=eigvals-efermi)
        f.create_dataset(f"e{efield}/eigvecs", data=eigvecs)
        f.create_dataset(f"e{efield}/vbm", data=vbm-efermi)
        f.create_dataset(f"e{efield}/cbm", data=cbm-efermi)
        f.create_dataset(f"e{efield}/efermi", data=efermi)


efield, efermi: -0.5 -1.2299713816153979
efield, efermi: -0.35 -0.16649843697765065
efield, efermi: -0.3 0.1939483756608003
efield, efermi: -0.2 0.9211341365594877
efield, efermi: 0 2.3825020706388162
efield, efermi: 0.2 3.768758576073611
efield, efermi: 0.3 4.4657253166413735
efield, efermi: 0.35 4.817502850291614
efield, efermi: 0.5 5.890618010607989
